In [27]:
# Homework 1:

# Using chinook.db write pandas code.

# 1. Customer Purchases Analysis:

# - Find the total amount spent by each customer on purchases (considering invoices).
# - Identify the top 5 customers with the highest total purchase amounts.
# - Display the customer ID, name, and the total amount spent for the top 5 customers.


import sqlite3
import pandas as pd

# Connect to the database
conn = sqlite3.connect("chinook.db")

# Query total purchase amount per customer
query = """ 
SELECT c.CustomerId AS Customer_ID, 
       COALESCE(c.FirstName, '') || ' ' || COALESCE(c.LastName, '') AS CustomerName,  
       SUM(i.Total) AS Total_Spent
FROM customers c
JOIN invoices i ON c.CustomerId = i.CustomerId
GROUP BY c.CustomerId
ORDER BY Total_Spent DESC
LIMIT 5;
"""

# Read the result into a DataFrame
top_customers = pd.read_sql(query, conn)

# Close the connection
conn.close()

# Display the result
print(top_customers)


   Customer_ID    CustomerName  Total_Spent
0           15   Benjamin King       400.75
1           13  William Walker       320.00
2           19    Henry Nelson       275.25
3           14    James Harris       250.00
4            7    David Wilson       230.00


In [26]:
# 2. Album vs. Individual Track Purchases:

# - Determine the percentage of customers who prefer to buy individual tracks instead of full albums.
# - A customer is considered to prefer individual tracks if they have purchased only a subset of tracks from an album.
# - Provide a summary of the percentage of customers who fall into each category (individual tracks vs. full albums).



import sqlite3
import pandas as pd

# Connect to SQLite database
conn = sqlite3.connect("chinook.db")

# Optimized SQL Query
query = """
with PurchaseAnalysis as (
    select i.CustomerId, t.AlbumId, COUNT(DISTINCT tp.TrackId) AS TracksPurchased,
           (select COUNT(*) FROM tracks WHERE AlbumId = t.AlbumId) AS TotalAlbumTracks
    FROM track_purchases tp
    JOIN invoices i ON tp.InvoiceId = i.InvoiceId
    JOIN tracks t ON tp.TrackId = t.TrackId
    GROUP BY i.CustomerId, t.AlbumId
)
SELECT 
    100.0 * SUM(TracksPurchased < TotalAlbumTracks) / COUNT(*) AS IndividualTrackPercentage,
    100.0 * SUM(TracksPurchased = TotalAlbumTracks) / COUNT(*) AS FullAlbumPercentage
FROM PurchaseAnalysis;
"""

# Execute the query and display results
summary = pd.read_sql(query, conn)
conn.close()

print(summary)

   IndividualTrackPercentage  FullAlbumPercentage
0                  77.777778            22.222222
